In [1]:
# Import required modules
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

# Check if CUDA is available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define a ResNet block with the Residual block change
class ResNet(nn.Module):
    def __init__(self, inchannels, outchannels, kernel_size=3, stride=1, skip=True):
        super().__init__()
        self.skip = skip
        self.block = nn.Sequential(
            nn.Conv2d(inchannels, outchannels, kernel_size=kernel_size, stride=stride, padding=1,bias=False),
            nn.BatchNorm2d(outchannels),
            nn.ReLU(),
            nn.Conv2d(outchannels, outchannels, kernel_size=kernel_size, padding=1,bias=False),
            nn.BatchNorm2d(outchannels),
            nn.ReLU(),
            nn.Conv2d(outchannels, outchannels, kernel_size=kernel_size, padding=1,bias=False),
            nn.BatchNorm2d(outchannels),
        )
        
        # If the input and output channels are different or the stride is 2, we need to apply a 1x1 convolution
        # to the input to match the output dimensions. This is called a skip connection.
        if stride == 2 or inchannels != outchannels:
            self.skip = False
            self.skip_conv = nn.Conv2d(inchannels, outchannels, kernel_size=1, stride=stride,bias=False)
            self.skip_bn = nn.BatchNorm2d(outchannels)
        
    def forward(self, x):
        out = self.block(x)
        if not self.skip:
            # If there is a skip connection, add it to the output of the block
            out += self.skip_bn(self.skip_conv(x))
        else:
            # Otherwise, add the input to the output of the block (identity mapping)
            out += x
        out = F.relu(out.clone())
        return out

# Define the ResNet architecture with multiple ResNet blocks
class ResNetF(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Define the initial convolution layer
        self.conv1 = nn.Conv2d(3, 32, kernel_size=7,stride=2, padding=3,bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=False)
        self.maxpool=nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        
        # Define multiple ResNet blocks with different input and output channels
        self.resblock1 = ResNet(32, 64,stride=1)
        self.resblock2 = ResNet(64, 64,stride=1)
        self.resblock3 = ResNet(64, 64,stride=1)
        self.resblock5=ResNet(64,64,stride=1)
        self.resblock6=ResNet(64,128,stride=2)
        self.resblock7=ResNet(128,128,stride=1)
        self.resblock10=ResNet(128,128,stride=1)
        self.resblock11=ResNet(128,256,stride=2)
        self.resblock12=ResNet(256,256,stride=1)
        
        # Define the final layers for classification
        self.avgpool=nn.AdaptiveAvgPool2d(output_size=(1, 1))
       


torch.Size([1, 10])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]           4,704
       BatchNorm2d-2         [-1, 32, 112, 112]              64
              ReLU-3         [-1, 32, 112, 112]               0
         MaxPool2d-4           [-1, 32, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          18,432
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11           [-1, 64, 56, 56]          36,864
      BatchNorm2d-12           [-1, 64, 56, 56]             128
           Conv2d-13           [-1, 64, 56, 56]           2,048
      BatchNorm2d-1

ResNetF(
  (conv1): Conv2d(3, 32, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (resblock1): ResNet(
    (block): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
      (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (skip_conv): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import numpy as np
from torch import nn
from torch.optim.lr_scheduler import CosineAnnealingLR


def normalize_image(image):
    image_min = image.min()
    image_max = image.max()
    image.clamp_(min = image_min, max = image_max)
    image.add_(-image_min).div_(image_max - image_min + 1e-5)
    return image

#initialize the model with kaiming_normal
def initialize_parameters(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight.data, nonlinearity = 'relu')
        nn.init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight.data, gain = nn.init.calculate_gain('relu'))
        nn.init.constant_(m.bias.data, 0)

def calculate_accuracy(y_pred, y): # calcualting the accuracy of the model
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc


#
#training the model the model
def train(model, iterator, optimizer, criterion, device): # traing the model on with the images in iterator
    
    epoch_loss = 0
    epoch_acc = 0
   
    model.train()
   
    for (x, y) in iterator:
        
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
                
        y_pred = model(x)
        
        loss = criterion(y_pred, y)
        
        acc = calculate_accuracy(y_pred, y)
        
        loss.backward()
        optimizer.step()
       
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    lr_scheduler.step()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


#evaluating the model

def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        
        for (x, y) in iterator:

            x = x.to(device)
            y = y.to(device)

            y_pred= model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [3]:
%%time

#transformations
transform = transforms.Compose(
    [
     transforms.RandomCrop(size=32, padding=4),
      transforms.RandomHorizontalFlip(),
     transforms.RandomRotation(20),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010]),
     ])

batch_size = 128

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                            shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True,transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010]),
                            ]))
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                            shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    
model =ResNetF()

optimizer = optim.SGD(model.parameters(), lr = 1e-1)

criterion = nn.CrossEntropyLoss()
# cosine annealing learning rate scheduler
lr_scheduler = CosineAnnealingLR(optimizer, 200)
model = model.to(device)
criterion = criterion.to(device)
  

loss =10
train_loss_history=[]
train_acc_history=[]
test_loss_history=[]
test_acc_history=[]
EPOCHS=200
for i in range(EPOCHS):
    # using train iterator
    train_loss , epoch_acc = train(model,trainloader, optimizer, criterion, device=device)
    print("train loss per epoch => ", i, train_loss, "train acc per epoch=> " , epoch_acc)
    # evaluating the results
    epoch_loss , epoch_valid_acc = evaluate(model, testloader, criterion, device)
    print("val loss per epoch => ", epoch_loss , "val acc per epoch =>",epoch_valid_acc,"\n")
        
    train_loss_history.append(train_loss)
    train_acc_history.append(epoch_acc)
    test_loss_history.append(epoch_loss)
    test_acc_history.append(epoch_valid_acc)
    
    if epoch_loss<loss:
        torch.save(model,"./res_block.pt")
        loss= epoch_loss


100.0%


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
train loss per epoch =>  0 2.121805954772188 train acc per epoch=>  0.3313419117647059
val loss per epoch =>  2.061382332934609 val acc per epoch => 0.3942840189873418 

train loss per epoch =>  1 2.0462236505030367 train acc per epoch=>  0.40917119566741805
val loss per epoch =>  2.0248137742658203 val acc per epoch => 0.43037974683544306 

train loss per epoch =>  2 2.0157427799976086 train acc per epoch=>  0.4408367966751918
val loss per epoch =>  1.9786574976353706 val acc per epoch => 0.478935917721519 

train loss per epoch =>  3 1.9940991108984594 train acc per epoch=>  0.46217631074168797
val loss per epoch =>  1.992312131048758 val acc per epoch => 0.46489319620253167 

train loss per epoch =>  4 1.9743512710342017 train acc per epoch=>  0.48233695652173914
val loss per epoch =>  1.9444244949123528 val acc per epoch => 0.5117681962025317 

train loss per epoch =>  5 1.9559028246213712 trai

In [4]:
max(test_acc_history) #get the max accuracy

0.8322784810126582

In [ ]:
#save the results of training
np.savetxt("./train_acc_history.txt",train_acc_history)
np.savetxt("./train_loss_history.txt",train_loss_history)
np.savetxt("./test_acc_history.txt",test_acc_history)
np.savetxt("./test_loss_history.txt",test_loss_history)